# Práctica 1 - Predicción de Subscripción a un Producto Bancario
**Miembros del grupo:** Christian Cano, Adrián Curell


## 1.EDA simplificado:
Lo primero que haremos será hacer el EDA del proyecto que nos permitirá entender, limpiar y visualizar la estructura de los datos antes de entrenar el modelo. Esto es fundamental para detectar errores, manejar valores nulos, identificar valores atípicos (outliers) y descubrir patrones o correlaciones ocultas que mejoran la precisión del modelo

### 1.1.Librerias:
Aquí importaremos todas las librerias que necesitaremos para hacer las comprobaciones del EDA, las cuales iremos nombrando segun se vayan necesitando. También creamos la semilla de reproducibilidad con uno de nuestros NIAs.

In [3]:
from google.colab import files
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Semilla de reproducibilidad
semilla = 100522320
np.random.seed(semilla)

### 1.2.Carga de datos:
Para acceder a bank_02.pkl primero tenemos que insertarlo al notebook. Esto la haremos con el uso de la biblioteca:

"from google.colab import files"

También usaremos pandas para tomarlo como dataframe.

In [4]:
# Subimos y leemos el pickle.
archivo = files.upload()
nombre_archivo=list(archivo.keys())[0]
with open(nombre_archivo,'rb') as f:
  datos=pickle.load(f)

df_datos = pd.DataFrame(datos)

Saving bank_02.pkl to bank_02.pkl


/tmp/ipython-input-428/2133411788.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  datos=pickle.load(f)


### 1.3.Numero de instancias y variables:
Con el uso de pandas podemos ver cuantas instancias y variables hay gracias a la función shape.

En este caso hay 11000 instancias y 17 variables.

In [5]:
# Determinar cuántas variables e instancias hay.
print(f'Dimensiones del dataset: {df_datos.shape}')
print(f'Número de instancias: {df_datos.shape[0]}')
print(f'Número de variables: {df_datos.shape[1]}')
df_datos.head()

Dimensiones del dataset: (11000, 17)
Número de instancias: 11000
Número de variables: 17


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,None,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


### 1.4.Tipo de las variables:
Lo primero que haremos será ver con pandas y su función dtypes el tipo de cada variable.

In [27]:
print(df_datos.dtypes.values)

[dtype('int64') dtype('O') dtype('O') dtype('O') dtype('O') dtype('int64')
 dtype('O') dtype('O') dtype('O') dtype('int64') dtype('O') dtype('int64')
 dtype('int64') dtype('int64') dtype('int64') dtype('O') dtype('O')]


Como podemos ver todas son o int64 o Object por lo que podemos hacer un clasificador automático que las separe en numéricas (int64) o categóricas (Object). Para realizarlo haremos:

1. Dividir las variables en numéricas o categóricas segun su dtype utilizando la función select_dtypes.

2. Crear 2 listas, una para el tipo (numéricas o categóricas) y otra para el rol (Target o Feature).

3. Recorrer el dataframe con un bucle y rellenando estas listas con los datos de las variables.

4. Crear un dataframe donde introduzcamos los datos de estas listas juntos al nombre y dtype de cada variable.

In [33]:
# Clasificación automática de variables
variables_numericas = df_datos.select_dtypes(include=[np.number]).columns.tolist()
variables_categoricas = df_datos.select_dtypes(include=['object']).columns.tolist()

target = 'deposit'

print('=== Variables NUMÉRICAS ===')
print(variables_numericas)

print('\n=== Variables CATEGÓRICAS ===')
print(variables_categoricas)

print(f'\nTarget: {target}')

# Creamos listas vacías donde almacenar los tipos y roles de cada variable
lista_tipo = []
lista_rol = []

# Recorrer columnas
for c in df_datos.columns:

    # Tipo (Numérica o Categórica)
    if c in variables_numericas:
        lista_tipo.append('Numérica')
    else:
        lista_tipo.append('Categórica')

    # Rol (Target o Feature)
    if c == target:
        lista_rol.append('Target')
    else:
        lista_rol.append('Feature')

# Crear DataFrame final
type_summary = pd.DataFrame({
    'Variable': df_datos.columns,
    'Dtype': df_datos.dtypes.values,
    'Tipo': lista_tipo,
    'Rol': lista_rol
})

print('\n', type_summary.to_string(index=True))

=== Variables NUMÉRICAS ===
['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

=== Variables CATEGÓRICAS ===
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']

Target: deposit

      Variable   Dtype        Tipo      Rol
0         age   int64    Numérica  Feature
1         job  object  Categórica  Feature
2     marital  object  Categórica  Feature
3   education  object  Categórica  Feature
4     default  object  Categórica  Feature
5     balance   int64    Numérica  Feature
6     housing  object  Categórica  Feature
7        loan  object  Categórica  Feature
8     contact  object  Categórica  Feature
9         day   int64    Numérica  Feature
10      month  object  Categórica  Feature
11   duration   int64    Numérica  Feature
12   campaign   int64    Numérica  Feature
13      pdays   int64    Numérica  Feature
14   previous   int64    Numérica  Feature
15   poutcome  object  Categórica  Feature
16    deposit  

### 1.5.Cardinalidad de variables categóricas
EXPLICACIÓN Y AÑADIR AL CODIGO PARA QUE MUESTRE LAS POSIBLES CATEGORIAS DE CADA VARIABLE

In [36]:
# Creamos listas vacías donde almacenar el numero de valores unicos de las variables categoricas
n_valores_unicos = []
valores_unicos = []



cardinalidad = pd.DataFrame({
    'Variable': variables_categoricas,
    'N_unique': [df_datos[c].nunique() for c in variables_categoricas],
    'Valores': [df_datos[c].unique().tolist() for c in variables_categoricas]
})
cardinalidad['Alta_cardinalidad (>10)'] = cardinalidad['N_unique'] > 10

print(cardinalidad[['Variable', 'N_unique', 'Alta_cardinalidad (>10)']].to_string(index=False))

high_card = cardinalidad[cardinalidad['Alta_cardinalidad (>10)']]['Variable'].tolist()
print(f'\nVariables con alta cardinalidad: {high_card if high_card else "Ninguna"}')

 Variable  N_unique  Alta_cardinalidad (>10)
      job        12                     True
  marital         3                    False
education         4                    False
  default         2                    False
  housing         2                    False
     loan         2                    False
  contact         3                    False
    month        12                     True
 poutcome         4                    False
  deposit         2                    False

Variables con alta cardinalidad: ['job', 'month']
